In [2]:
import numpy as np

import ng_utils
from sofima.zarr import zarr_io

Datasets to register: 
- diSPIM_685890_2023-06-29_14-39-56
https://aind-neuroglancer-sauujisjxq-uw.a.run.app#!s3://aind-open-data/diSPIM_685890_2023-06-29_14-39-56/process_output.json

- diSPIM_685890_2023-06-30_14-13-22
https://aind-neuroglancer-sauujisjxq-uw.a.run.app#!s3://aind-open-data/diSPIM_685890_2023-06-30_14-13-22/process_output.json

- diSPIM_685890_2023-07-04_12-50-25
https://aind-neuroglancer-sauujisjxq-uw.a.run.app#!s3://aind-open-data/diSPIM_685890_2023-07-04_12-50-25/process_output.json

Carson: 
- diSPIM_672347-ID_2023-07-13_14-58-59


In [15]:
cloud_storage = zarr_io.CloudStorage.S3
bucket = 'aind-open-data'
dataset = 'diSPIM_672347-ID_2023-07-13_14-58-59'
dataset_path = f'{dataset}/diSPIM.zarr'
channel = 561
downsample_exp = 0

# These datasets are cam1 and axis flipped. 
zd = zarr_io.DiSpimDataset(cloud_storage, 
                        bucket, 
                        dataset_path, 
                        channel, 
                        downsample_exp, 
                        camera_num=1)

# Pixel overlap offsets in highest resolution
# We want to parse this from the metadata files eventually
ty, tx = zd.tile_layout.shape
mx, my, mz = zd.tile_size_xyz
coarse_mesh = np.zeros((3, 1, ty, tx))
x_overlap = 0.15
y_overlap = 0.15
for y in range(ty):
    for x in range(tx):
        if y > 0: 
            coarse_mesh[:, 0, y, x] += np.array([0, (my * y_overlap), 0])
        if x > 0: 
            coarse_mesh[:, 0, y, x] += np.array([(mx * x_overlap), 0, 0])

        # coarse_mesh[:, 0, y, x] = np.array([(mx * x_overlap), 0, 0]) + \
        #                           np.array([0, (my * y_overlap), 0])
        coarse_mesh[:, 0, y, x] = zd.basis_change @ coarse_mesh[:, 0, y, x]

# Generate neuroglancer link: 
# zd_list = [zd]
# remaining_channels = list(set(zd.channels) - set([channel]))
# for channel in remaining_channels: 
#     zd_list.append(zarr_io.DiSpimDataset(cloud_storage, 
#                                         bucket, 
#                                         dataset_path, 
#                                         channel, 
#                                         downsample_exp))
# ng_utils.ng_link_multi_channel(zd_list, coarse_mesh, 
#                                max_dr=800, opacity=0.5)


ng_utils.ng_link_single_channel(zd, coarse_mesh, 
                               max_dr=800, opacity=0.5)


Looks like data is cam1. 
